In [42]:
!python -m pip install requests
!python -m pip install imdbpy
!python -m pip install pandas
!python -m pip install tmdbsimple

In [43]:
#
# from google.colab import drive
# drive.mount('/content/drive')

### This will only run one time to get the information from the dataset: https://www.kaggle.com/datasets/akshaypawar7/millions-of-moviesw

In [44]:
# import pandas as pd
# import tmdbsimple as tmdb
# from concurrent.futures import ThreadPoolExecutor, as_completed

# # Set up TMDB API credentials
# tmdb.API_KEY = '9b023caa1c3becd6d577ab0e41927545'

# # Define function to get director information from TMDB API
# def get_director_info(movie_id):
#     url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={tmdb.API_KEY}"
#     response = requests.get(url)
#     try:
#         response.raise_for_status()
#         movie_info = response.json()
#         if 'crew' in movie_info:
#             for crew_member in movie_info['crew']:
#                 if crew_member['job'] == 'Director':
#                     return {'id': crew_member['id'], 'name': crew_member['name']}
#     except requests.exceptions.HTTPError as e:
#         if e.response.status_code == 404:
#             print(f"Movie with ID {movie_id} not found.")
#         else:
#             raise e
#     return None

# # Define a function to process a chunk of the movies DataFrame
# def process_chunk(chunk):
#     # Keep only relevant columns using .loc
#     chunk = chunk.loc[:, ['id', 'title', 'release_date', 'budget', 'revenue', 'credits']]
#     # Get director information for each movie in the chunk
#     directors = []
#     for index, row in chunk.iterrows():
#         movie_id = row['id']
#         director = get_director_info(movie_id)
#         if director is not None:
#             directors.append(director['name'])
#         else:
#             directors.append(None)
#     # Add director column to the chunk using .loc
#     chunk.loc[:, 'director'] = directors
#     return chunk

# # Read movies.csv into a pandas dataframe in chunks, starting from row 350000
# chunk_size = 10000
# start_row = 0
# #Change the location to your file location
# df_iter = pd.read_csv('/content/drive/MyDrive/movies.csv', chunksize=chunk_size, skiprows=range(1, start_row+1))

# # Use multithreading to process the chunks
# output_file = '/content/drive/MyDrive/movies_with_directors.csv'
# with ThreadPoolExecutor() as executor:
#     futures = []
#     for i, chunk in enumerate(df_iter):
#         futures.append(executor.submit(process_chunk, chunk))
#         print(f"Submitted chunk {i}")
#     # Write the processed chunks to the output CSV file as they complete
#     for future in as_completed(futures):
#         chunk = future.result()
#         chunk.to_csv(output_file, mode='a', header=False, index=False)

### Getting new movie information from the Movie Database (TMDb) API

In [45]:
import requests
import pandas as pd
from pandas import json_normalize

tmdb_api_key = '9b023caa1c3becd6d577ab0e41927545'

def get_movie_info(movie_id):
    # Make a request to the TMDb API for the movie information
   # https://api.themoviedb.org/3/movie/502356?api_key=9b023caa1c3becd6d577ab0e41927545&language=en-US 
    url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}&language=en-US'
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Retrieve the relevant information from the response JSON
        movie_info = response.json()
        return movie_info
    else:
        print(f"Request failed with status code {response.status_code}")

# Testing
movie_id = '760348' 
movie_dict = get_movie_info(movie_id)
flat_movie_dict = json_normalize(movie_dict)
movies_df = pd.DataFrame(flat_movie_dict)
movies_df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/wrvnGcLxmGTlgnZyQyKYaTnibdh.jpg,None,500,"[{'id': 10402, 'name': 'Music'}]",http://harjaspreetsingh.com/projects/gal-dil-di/,760348,tt13394078,pa,Gal Dil Di,...,2020-11-03,700,4,"[{'english_name': 'Punjabi', 'iso_639_1': 'pa'...",Released,,Gal Dil Di,True,0.0,0


In [46]:
# Retrieve the newest 100 movies that being released

def get_new_movies(num_movies_to_get):
  tmdb_api_key = '9b023caa1c3becd6d577ab0e41927545'
  BASE_URL = 'https://api.themoviedb.org/3/movie/now_playing'

  movies = []
  page = 1

  while len(movies) < num_movies_to_get:
      response = requests.get(BASE_URL, params={'api_key': tmdb_api_key, 'page': page})
      if response.status_code == 200:
          data = response.json()
          for movie in data['results']:
              movie_info = get_movie_info(movie['id'])
              movies.append(movie_info)
              if len(movies) >= 100:
                  break
          page += 1
      else:
          break
      return movies

new_movies_df = pd.DataFrame(get_new_movies(100))
new_movies_df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/9n2tJBplPbgR2ca05hS5CKXwP2c.jpg,None,100000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",https://www.thesupermariobros.movie,502356,tt6718170,en,The Super Mario Bros. Movie,...,2023-04-05,891438061,92,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,The Super Mario Bros. Movie,False,7.500,1512
1,False,/wybmSmviUXxlBmX44gtpow5Y9TB.jpg,"{'id': 724848, 'name': 'Shazam! Collection', '...",125000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",,594767,tt10151854,en,Shazam! Fury of the Gods,...,2023-03-15,132128334,130,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Oh. My. Gods.,Shazam! Fury of the Gods,False,6.867,1283
2,False,/7bWxAsNPv9CXHOhZbJVlj2KxgfP.jpg,None,12000000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",https://www.evildeadrisemovie.com,713704,tt13345606,en,Evil Dead Rise,...,2023-04-12,40300000,96,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Mommy loves you to death.,Evil Dead Rise,False,6.934,226
3,False,/ouB7hwclG7QI3INoYJHaZL4vOaa.jpg,"{'id': 94602, 'name': 'Puss in Boots Collectio...",90000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",https://www.dreamworks.com/movies/puss-in-boot...,315162,tt3915174,en,Puss in Boots: The Last Wish,...,2022-12-07,480000000,103,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Say hola to his little friends.,Puss in Boots: The Last Wish,False,8.296,5349
4,False,/h8gHn0OzBoaefsYseUByqsmEDMY.jpg,"{'id': 404609, 'name': 'John Wick Collection',...",90000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",https://johnwick.movie,603692,tt10366206,en,John Wick: Chapter 4,...,2023-03-22,362712705,170,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,No way back. One way out.,John Wick: Chapter 4,False,7.960,1227
5,False,/44immBwzhDVyjn87b3x3l9mlhAD.jpg,"{'id': 2602, 'name': 'Scream Collection', 'pos...",35000000,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",https://www.screammovie.com,934433,tt17663992,en,Scream VI,...,2023-03-08,168541093,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,New York. New rules.,Scream VI,False,7.400,764
6,False,/54IXMMEQKlkPXHqPExWy98UBmtE.jpg,None,0,"[{'id': 27, 'name': 'Horror'}]",,1008005,tt18072540,es,La niña de la comunión,...,2023-02-10,0,103,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,,The Communion Girl,False,6.500,58
7,False,/a2tys4sD7xzVaogPntGsT1ypVoT.jpg,None,35000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 35, 'n...",https://www.cocainebear.movie/,804150,tt14209916,en,Cocaine Bear,...,2023-02-22,80000000,96,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Get in line.,Cocaine Bear,False,6.398,887
8,False,/eEF40Xk2twM3WjRNZftfo771gjv.jpg,None,45000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.65.movie,700391,tt12261776,en,65,...,2023-03-02,55000000,93,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"65 million years ago, prehistoric Earth had a ...",65,False,6.307,786
9,False,/5Y5pz0NX7SZS9036I733F7uNcwK.jpg,"{'id': 1118596, 'name': 'The Pope's Exorcist',...",18000000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",https://www.thepopes-exorcist.movie,758323,tt13375076,en,The Pope's Exorcist,...,2023-04-05,52000000,103,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Inspired by the actual files of Father Gabriel...,The Pope's Exorcist,False,6.536,151


In [47]:
new_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  20 non-null     bool   
 1   backdrop_path          20 non-null     object 
 2   belongs_to_collection  7 non-null      object 
 3   budget                 20 non-null     int64  
 4   genres                 20 non-null     object 
 5   homepage               20 non-null     object 
 6   id                     20 non-null     int64  
 7   imdb_id                20 non-null     object 
 8   original_language      20 non-null     object 
 9   original_title         20 non-null     object 
 10  overview               20 non-null     object 
 11  popularity             20 non-null     float64
 12  poster_path            20 non-null     object 
 13  production_companies   20 non-null     object 
 14  production_countries   20 non-null     object 
 15  release_

## Clean up the new movies data for training



In [48]:
import imdb
ia = imdb.IMDb()

def get_imdb_movie_info(imdb_id):
    # Search for the movie by ID
    movie = ia.get_movie(imdb_id)
    # Retrieve all available information
    info_dict = movie.data
    return info_dict

movie_info = get_imdb_movie_info('0113041')
movie_info_json = json_normalize(movie_info)

movie_info

{'localized title': 'Father of the Bride Part II',
 'cast': [<Person id:0000188[http] name:_Steve Martin_>,
  <Person id:0000473[http] name:_Diane Keaton_>,
  <Person id:0001737[http] name:_Martin Short_>,
  <Person id:0931090[http] name:_Kimberly Williams-Paisley_>,
  <Person id:0627624[http] name:_George Newbern_>,
  <Person id:0001085[http] name:_Kieran Culkin_>,
  <Person id:0000703[http] name:_BD Wong_>,
  <Person id:0324533[http] name:_Peter Michael Goetz_>,
  <Person id:0570033[http] name:_Kate McGregor-Stewart_>,
  <Person id:0011038[http] name:_Jane Adams_>,
  <Person id:0506405[http] name:_Eugene Levy_>,
  <Person id:0150427[http] name:_Rebecca Chambers_>,
  <Person id:0651074[http] name:_April Ortiz_>,
  <Person id:0736851[http] name:_Dulcy Rogers_>,
  <Person id:0030926[http] name:_Kathy Anthony_>,
  <Person id:0134852[http] name:_Adrian Canzoneri_>,
  <Person id:0015935[http] name:_Lori Alan_>,
  <Person id:0589353[http] name:_Stephanie Miller_>,
  <Person id:0583654[http]

In [49]:
box_office = movie_info.get('director')
test = movie_info.get('runtimes')
test

['106']

In [50]:
results = ia.search_movie('Avater')
# Get the first result
movie = results[0]
movie

<Movie id:1630029[http] title:_Avatar: The Way of Water (None)_>

In [51]:
import pandas as pd
import re

# Loop through each row in the DataFrame
def fill_unknown_rows(new_movies_df):
    if 'director' not in new_movies_df.columns:
        new_movies_df['director'] = None
    if 'cast' not in new_movies_df.columns:
        new_movies_df['cast'] = None

    # Create a list to store the rows that need to be updated
    rows_to_update = []

    for i, row in new_movies_df.iterrows():
        # Check if any values are missing or if budget/revenue is zero
        if row.isnull().any() or row['budget'] == 0 or row['revenue'] == 0:
            # If so, get the IMDB info for that movie
            try:
              imdb_id = re.sub('^tt', '', row['imdb_id']) # Remove 'tt' prefix
              imdb_info = get_imdb_movie_info(imdb_id)
              # Append the row index and IMDB info to the list
              rows_to_update.append((i, imdb_info))
            except:
              pass

    # Update the DataFrame with the new info for the selected rows
    for row_index, imdb_info in rows_to_update:
        row = new_movies_df.loc[row_index]
        actor_list = imdb_info.get('cast')
        actor_names = [person['name'] for person in actor_list[:5]]

        new_movies_df.at[row_index, 'original title'] = imdb_info.get('original title')
        new_movies_df.at[row_index, 'budget'] = imdb_info.get('box office', {}).get('Budget')
        new_movies_df.at[row_index, 'revenue'] = imdb_info.get('box office', {}).get('Cumulative Worldwide Gross')
        new_movies_df.at[row_index, 'director'] = imdb_info.get('director', [{}])[0].get('name')
        new_movies_df.at[row_index, 'rating'] = imdb_info.get('rating')
        new_movies_df.at[row_index, 'genres'] = imdb_info.get('genres')[0]
        new_movies_df.at[row_index, 'cast'] = actor_names
        new_movies_df.at[row_index, 'vote_count'] = imdb_info.get('votes')
        new_movies_df.at[row_index, 'runtimes'] = imdb_info.get('runtimes')

    return new_movies_df


In [52]:
new_movies_df.isnull().sum()

adult                     0
backdrop_path             0
belongs_to_collection    13
budget                    0
genres                    0
homepage                  0
id                        0
imdb_id                   0
original_language         0
original_title            0
overview                  0
popularity                0
poster_path               0
production_companies      0
production_countries      0
release_date              0
revenue                   0
runtime                   0
spoken_languages          0
status                    0
tagline                   0
title                     0
video                     0
vote_average              0
vote_count                0
dtype: int64

In [53]:
new_movies_df_filled=fill_unknown_rows(new_movies_df)
new_movies_df_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  20 non-null     bool   
 1   backdrop_path          20 non-null     object 
 2   belongs_to_collection  7 non-null      object 
 3   budget                 12 non-null     object 
 4   genres                 20 non-null     object 
 5   homepage               20 non-null     object 
 6   id                     20 non-null     int64  
 7   imdb_id                20 non-null     object 
 8   original_language      20 non-null     object 
 9   original_title         20 non-null     object 
 10  overview               20 non-null     object 
 11  popularity             20 non-null     float64
 12  poster_path            20 non-null     object 
 13  production_companies   20 non-null     object 
 14  production_countries   20 non-null     object 
 15  release_

In [54]:
new_movies_df_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  20 non-null     bool   
 1   backdrop_path          20 non-null     object 
 2   belongs_to_collection  7 non-null      object 
 3   budget                 12 non-null     object 
 4   genres                 20 non-null     object 
 5   homepage               20 non-null     object 
 6   id                     20 non-null     int64  
 7   imdb_id                20 non-null     object 
 8   original_language      20 non-null     object 
 9   original_title         20 non-null     object 
 10  overview               20 non-null     object 
 11  popularity             20 non-null     float64
 12  poster_path            20 non-null     object 
 13  production_companies   20 non-null     object 
 14  production_countries   20 non-null     object 
 15  release_

In [55]:
import pandas as pd
import ast

# Load the two datasets from the old kaggle
movies_file = "movie_data/movies_metadata.csv"
credits_file = "movie_data/credits.csv"

df_movies = pd.read_csv(movies_file, dtype={'id': 'object'})
df_credits = pd.read_csv(credits_file, dtype={'id': 'object', 'cast': 'object', 'crew': 'str'})

# Merge the two datasets on "id"
df_merged = pd.merge(df_movies, df_credits, on='id')

# Define a function to extract the directors
def get_directors(crew_list):
    directors = []
    for crew_member in ast.literal_eval(crew_list):
        if crew_member['job'] == 'Director':
            directors.append(crew_member['name'])
    return directors

# Define a function to extract the cast
def get_cast(cast_list):
    cast = []
    for cast_member in ast.literal_eval(cast_list):
        cast.append(cast_member['name'])
    return cast

# Apply the functions to extract the directors and cast
df_merged['director'] = df_merged['crew'].apply(get_directors)
df_merged['cast'] = df_merged['cast'].apply(get_cast)
df_merged = df_merged.drop('crew', axis=1)

C:\Users\hongj\AppData\Local\Temp\ipykernel_5940\947431000.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(movies_file, dtype={'id': 'object'})


In [56]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45538 entries, 0 to 45537
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45538 non-null  object 
 1   belongs_to_collection  4500 non-null   object 
 2   budget                 45538 non-null  object 
 3   genres                 45538 non-null  object 
 4   homepage               7792 non-null   object 
 5   id                     45538 non-null  object 
 6   imdb_id                45521 non-null  object 
 7   original_language      45527 non-null  object 
 8   original_title         45538 non-null  object 
 9   overview               44584 non-null  object 
 10  popularity             45535 non-null  object 
 11  poster_path            45152 non-null  object 
 12  production_companies   45535 non-null  object 
 13  production_countries   45535 non-null  object 
 14  release_date           45451 non-null  object 
 15  re

In [57]:
import pandas as pd

df_movies_tmdb = pd.read_csv("movie_data/movies_with_directors.csv", dtype={'id': 'object'})
df_movies_tmdb = df_movies_tmdb[['id', 'title', 'release_date', 'budget', 'revenue', 'credits', 'director']]
df_movies_tmdb = df_movies_tmdb.rename(columns={
    'title': 'original_title',
    'credits': 'cast'
})

more_movies_file = "movie_data/movies.csv"
df_movies_larger = pd.read_csv(more_movies_file, dtype={'id': 'object'})

# Merge the 'director' column from df_movies_tmdb into df_movies_larger based on 'id'
df_movies_larger = df_movies_larger.merge(df_movies_tmdb[['id', 'director']], on='id', how='left')
df_movies_larger = df_movies_larger.rename(columns={
    'title': 'original_title',
    'credits': 'cast'
})
# Print the updated dataframe
df_movies_larger


,id,original_title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,...,status,tagline,vote_average,vote_count,cast,keywords,poster_path,backdrop_path,recommendations,director
0,677179,Creed III,Drama-Action,en,After dominating the boxing world Adonis Creed...,9575.225,Metro-Goldwyn-Mayer-Proximity Media-Balboa Pro...,2023-03-01,75000000.0,2.580000e+08,...,Released,You can't run from your past.,7.305,845.0,Michael B. Jordan-Tessa Thompson-Jonathan Majo...,philadelphia pennsylvania-husband wife relatio...,/vJU3rXSP9hwUuLeq8IpfsJShLOk.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-107...,Michael B. Jordan
1,76600,Avatar: The Way of Water,Science Fiction-Adventure-Action,en,Set more than a decade after the events of the...,9366.788,20th Century Studios-Lightstorm Entertainment,2022-12-14,350000000.0,2.312336e+09,...,Released,Return to Pandora.,7.751,6748.0,Sam Worthington-Zoe Saldaña-Sigourney Weaver-S...,loss of loved one-dying and death-alien life-f...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg,183392-111332-702432-505642-1064215-436270-874...,James Cameron
2,502356,The Super Mario Bros. Movie,Animation-Adventure-Family-Fantasy-Comedy,en,While working underground to fix a water main ...,5132.098,Universal Pictures-Illumination-Nintendo,2023-04-05,100000000.0,5.800000e+07,...,Released,NaN,7.556,332.0,Chris Pratt-Anya Taylor-Joy-Charlie Day-Jack B...,video game-plumber-magic mushroom-based on vid...,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,/iw0Na1UBHgA5BgifwmQ8vKhlWgA.jpg,NaN,Michael Jelenic
3,631842,Knock at the Cabin,Horror-Mystery-Thriller,en,While vacationing at a remote cabin a young gi...,3422.537,Blinding Edge Pictures-Universal Pictures-Film...,2023-02-01,20000000.0,5.200000e+07,...,Released,Save your family or save humanity. Make the ch...,6.457,888.0,Dave Bautista-Jonathan Groff-Ben Aldridge-Kris...,based on novel or book-sacrifice-cabin-faith-e...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-104...,M. Night Shyamalan
4,646389,Plane,Action-Adventure-Thriller,en,After a heroic job of successfully landing his...,2618.646,MadRiver Pictures-Di Bonaventura Pictures-G-BA...,2023-01-12,25000000.0,5.100000e+07,...,Released,Survive together or die alone.,6.901,785.0,Gerard Butler-Mike Colter-Yoson An-Tony Goldwy...,pilot-airplane-philippines-held hostage-plane ...,/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg,/9Rq14Eyrf7Tu1xk0Pl7VcNbNh1n.jpg,505642-758769-864692-631842-1058949-925943-758...,Jean-François Richet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855565,887860,拯救校花计划,Drama,zh,NaN,0.600,NaN,2017-12-02,0.0,0.000000e+00,...,Released,NaN,0.000,0.0,NaN,NaN,/2LXG2Zhp2diqM0qLNzhZ4zbQvof.jpg,/cE4gpxY3OASBGPpv0fQqHWL4dMd.jpg,NaN,NaN
855566,886714,Oberhausen 34. Kurzfilmtage,NaN,de,01.04.1988,0.600,NaN,1988-04-01,0.0,0.000000e+00,...,Released,NaN,0.000,0.0,NaN,NaN,/yevFfrQxkwJwMMhEJqDrJKgXFp.jpg,NaN,NaN,Schmelzdahin
855567,774222,Hader fürs Heim,NaN,de,Josef Hader's first television broadcast a mix...,0.600,NaN,1991-01-01,0.0,0.000000e+00,...,Released,NaN,0.000,0.0,Josef Hader-Maria Hofstätter-Alfred Dorfer,NaN,/hBupAGQkT1yPdwZUXcxksxUsGkr.jpg,NaN,NaN,NaN
855568,832174,Into The Light : A Trip Through Baltimore Ball...,NaN,en,A not-so-Cinderella-story of LGBTQ+ Baltimore ...,0.600,NaN,2021-05-21,0.0,0.000000e+00,...,Released,NaN,0.000,0.0,Keith Holt-Londyn Smith de Richelieu-Marquis C...,NaN,/h3gUEQ2CLMdbv95l5Sxd2VDG0Bx.jpg,NaN,NaN,Jason Gray


In [58]:
df_movies_larger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855570 entries, 0 to 855569
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    855570 non-null  object 
 1   original_title        855564 non-null  object 
 2   genres                620773 non-null  object 
 3   original_language     855570 non-null  object 
 4   overview              721075 non-null  object 
 5   popularity            855570 non-null  float64
 6   production_companies  411577 non-null  object 
 7   release_date          799735 non-null  object 
 8   budget                855570 non-null  float64
 9   revenue               855570 non-null  float64
 10  runtime               816628 non-null  float64
 11  status                855570 non-null  object 
 12  tagline               131167 non-null  object 
 13  vote_average          855570 non-null  float64
 14  vote_count            855570 non-null  float64
 15  

In [59]:
merged_df = pd.merge(df_merged, df_movies_larger, on='id', how='outer')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859677 entries, 0 to 859676
Data columns (total 46 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   adult                   51657 non-null   object 
 1   belongs_to_collection   4626 non-null    object 
 2   budget_x                51657 non-null   object 
 3   genres_x                51657 non-null   object 
 4   homepage                9324 non-null    object 
 5   id                      859677 non-null  object 
 6   imdb_id                 51640 non-null   object 
 7   original_language_x     51643 non-null   object 
 8   original_title_x        51657 non-null   object 
 9   overview_x              50483 non-null   object 
 10  popularity_x            51654 non-null   object 
 11  poster_path_x           51161 non-null   object 
 12  production_companies_x  51654 non-null   object 
 13  production_countries    51654 non-null   object 
 14  release_date_x      

In [60]:
columns_to_merge = ['belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language',
                    'original_title', 'overview', 'popularity', 'poster_path', 'production_companies',
                    'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
                    'tagline', 'video', 'vote_average', 'vote_count', 'cast', 'director']

for column in columns_to_merge:
    x_column = f"{column}_x"
    y_column = f"{column}_y"

    if x_column in merged_df.columns and y_column in merged_df.columns:
        merged_df[column] = merged_df[y_column].combine_first(merged_df[x_column])
        merged_df = merged_df.drop(columns=[x_column, y_column])
    elif x_column in merged_df.columns:
        merged_df = merged_df.rename(columns={x_column: column})
    elif y_column in merged_df.columns:
        merged_df = merged_df.rename(columns={y_column: column})


In [61]:
merged_df

,adult,belongs_to_collection,homepage,id,imdb_id,production_countries,spoken_languages,title,video,keywords,...,production_companies,release_date,revenue,runtime,status,tagline,vote_average,vote_count,cast,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",http://toystory.disney.com/toy-story,862,tt0114709,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,False,martial arts-jealousy-friendship-bullying-elem...,...,Pixar,1995-10-30,373554033.0,81.0,Released,NaN,7.966,16430.0,Tom Hanks-Tim Allen-Don Rickles-Jim Varney-Wal...,John Lasseter
1,False,NaN,NaN,8844,tt0113497,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,False,giant insect-board game-jungle-disappearance-r...,...,TriStar Pictures-PolyGram Filmed Entertainment...,1995-12-15,262821940.0,104.0,Released,Roll the dice and unleash the excitement!,7.237,9437.0,Robin Williams-Kirsten Dunst-Bradley Pierce-Bo...,Joe Johnston
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,15602,tt0113228,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,False,fishing-halloween-sequel-old man-best friend-w...,...,Lancaster Gate-Warner Bros. Pictures,1995-12-22,71500000.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,6.500,324.0,Walter Matthau-Jack Lemmon-Ann-Margret-Sophia ...,Howard Deutch
3,False,NaN,NaN,31357,tt0114885,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,False,based on novel or book-interracial relationshi...,...,20th Century Fox,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,6.300,128.0,Whitney Houston-Angela Bassett-Loretta Devine-...,Forest Whitaker
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,11862,tt0113041,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,False,parent child relationship-baby-midlife crisis-...,...,Sandollar Productions-Touchstone Pictures,1995-12-08,76594107.0,106.0,Released,Just When His World Is Back To Normal... He's ...,6.200,626.0,Steve Martin-Diane Keaton-Martin Short-Kimberl...,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859672,NaN,NaN,NaN,887860,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-12-02,0.0,0.0,Released,NaN,0.000,0.0,NaN,NaN
859673,NaN,NaN,NaN,886714,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1988-04-01,0.0,3.0,Released,NaN,0.000,0.0,NaN,Schmelzdahin
859674,NaN,NaN,NaN,774222,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1991-01-01,0.0,40.0,Released,NaN,0.000,0.0,Josef Hader-Maria Hofstätter-Alfred Dorfer,NaN
859675,NaN,NaN,NaN,832174,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2021-05-21,0.0,25.0,Released,NaN,0.000,0.0,Keith Holt-Londyn Smith de Richelieu-Marquis C...,Jason Gray


In [62]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859677 entries, 0 to 859676
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   adult                  51657 non-null   object 
 1   belongs_to_collection  4626 non-null    object 
 2   homepage               9324 non-null    object 
 3   id                     859677 non-null  object 
 4   imdb_id                51640 non-null   object 
 5   production_countries   51654 non-null   object 
 6   spoken_languages       51654 non-null   object 
 7   title                  51654 non-null   object 
 8   video                  51654 non-null   object 
 9   keywords               255914 non-null  object 
 10  backdrop_path          277217 non-null  object 
 11  recommendations        40532 non-null   object 
 12  budget                 859677 non-null  object 
 13  genres                 625538 non-null  object 
 14  original_language      859677 non-nu

In [63]:
merged_df_with_valid_box_office = merged_df[(merged_df['budget'].notnull()) & (merged_df['revenue'] >= 100)]
merged_df_with_valid_box_office

,adult,belongs_to_collection,homepage,id,imdb_id,production_countries,spoken_languages,title,video,keywords,...,production_companies,release_date,revenue,runtime,status,tagline,vote_average,vote_count,cast,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",http://toystory.disney.com/toy-story,862,tt0114709,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,False,martial arts-jealousy-friendship-bullying-elem...,...,Pixar,1995-10-30,373554033.0,81.0,Released,NaN,7.966,16430.0,Tom Hanks-Tim Allen-Don Rickles-Jim Varney-Wal...,John Lasseter
1,False,NaN,NaN,8844,tt0113497,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,False,giant insect-board game-jungle-disappearance-r...,...,TriStar Pictures-PolyGram Filmed Entertainment...,1995-12-15,262821940.0,104.0,Released,Roll the dice and unleash the excitement!,7.237,9437.0,Robin Williams-Kirsten Dunst-Bradley Pierce-Bo...,Joe Johnston
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,15602,tt0113228,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,False,fishing-halloween-sequel-old man-best friend-w...,...,Lancaster Gate-Warner Bros. Pictures,1995-12-22,71500000.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,6.500,324.0,Walter Matthau-Jack Lemmon-Ann-Margret-Sophia ...,Howard Deutch
3,False,NaN,NaN,31357,tt0114885,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,False,based on novel or book-interracial relationshi...,...,20th Century Fox,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,6.300,128.0,Whitney Houston-Angela Bassett-Loretta Devine-...,Forest Whitaker
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,11862,tt0113041,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,False,parent child relationship-baby-midlife crisis-...,...,Sandollar Productions-Touchstone Pictures,1995-12-08,76594107.0,106.0,Released,Just When His World Is Back To Normal... He's ...,6.200,626.0,Steve Martin-Diane Keaton-Martin Short-Kimberl...,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857900,NaN,NaN,NaN,791261,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,50000000.0,124.0,Released,Boring.,0.000,0.0,NaN,NaN
858502,NaN,NaN,NaN,783065,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,150.0,0.0,Released,NaN,0.000,0.0,NaN,NaN
858622,NaN,NaN,NaN,757237,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2018-02-13,500.0,0.0,Released,Russell and Sarah are faced with what to do af...,0.000,0.0,NaN,NaN
858829,NaN,NaN,NaN,830358,NaN,NaN,NaN,NaN,NaN,trauma-turkey-based on true story-nature run a...,...,NaN,2016-04-01,720.0,2.0,Released,"« Je criais: ""Y'a une bibitte noire dans le sa...",10.000,1.0,NaN,NaN


In [64]:
merged_df_with_valid_box_office['Top Cast'] = ''

for index, row in merged_df_with_valid_box_office.iterrows():
    try:
        cast_names = row['cast'].split('-')
    except:
        cast_names = row['cast']
    if cast_names is not None and isinstance(cast_names, list) and len(cast_names) > 0:
        top_cast_names = cast_names[:5]
        merged_df_with_valid_box_office.at[index, 'Top Cast'] = top_cast_names


C:\Users\hongj\AppData\Local\Temp\ipykernel_5940\2095058402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_with_valid_box_office['Top Cast'] = ''


In [65]:
merged_df_with_valid_box_office

,adult,belongs_to_collection,homepage,id,imdb_id,production_countries,spoken_languages,title,video,keywords,...,release_date,revenue,runtime,status,tagline,vote_average,vote_count,cast,director,Top Cast
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",http://toystory.disney.com/toy-story,862,tt0114709,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,False,martial arts-jealousy-friendship-bullying-elem...,...,1995-10-30,373554033.0,81.0,Released,NaN,7.966,16430.0,Tom Hanks-Tim Allen-Don Rickles-Jim Varney-Wal...,John Lasseter,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,False,NaN,NaN,8844,tt0113497,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,False,giant insect-board game-jungle-disappearance-r...,...,1995-12-15,262821940.0,104.0,Released,Roll the dice and unleash the excitement!,7.237,9437.0,Robin Williams-Kirsten Dunst-Bradley Pierce-Bo...,Joe Johnston,"[Robin Williams, Kirsten Dunst, Bradley Pierce..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,15602,tt0113228,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,False,fishing-halloween-sequel-old man-best friend-w...,...,1995-12-22,71500000.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,6.500,324.0,Walter Matthau-Jack Lemmon-Ann-Margret-Sophia ...,Howard Deutch,"[Walter Matthau, Jack Lemmon, Ann, Margret, So..."
3,False,NaN,NaN,31357,tt0114885,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,False,based on novel or book-interracial relationshi...,...,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,6.300,128.0,Whitney Houston-Angela Bassett-Loretta Devine-...,Forest Whitaker,"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,11862,tt0113041,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,False,parent child relationship-baby-midlife crisis-...,...,1995-12-08,76594107.0,106.0,Released,Just When His World Is Back To Normal... He's ...,6.200,626.0,Steve Martin-Diane Keaton-Martin Short-Kimberl...,Charles Shyer,"[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857900,NaN,NaN,NaN,791261,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,50000000.0,124.0,Released,Boring.,0.000,0.0,NaN,NaN,
858502,NaN,NaN,NaN,783065,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,150.0,0.0,Released,NaN,0.000,0.0,NaN,NaN,
858622,NaN,NaN,NaN,757237,NaN,NaN,NaN,NaN,NaN,NaN,...,2018-02-13,500.0,0.0,Released,Russell and Sarah are faced with what to do af...,0.000,0.0,NaN,NaN,
858829,NaN,NaN,NaN,830358,NaN,NaN,NaN,NaN,NaN,trauma-turkey-based on true story-nature run a...,...,2016-04-01,720.0,2.0,Released,"« Je criais: ""Y'a une bibitte noire dans le sa...",10.000,1.0,NaN,NaN,


In [66]:
unique_cast_count = merged_df_with_valid_box_office['Top Cast'].explode().nunique()
unique_cast_count

32238

In [67]:
# merged_df_with_valid_box_office_filled=fill_unknown_rows(merged_df_with_valid_box_office)



In [68]:
# merged_df_with_valid_box_office_filled

In [69]:
new_movies_df_filled['id'] = new_movies_df_filled['id'].astype(str)
# Check if ID already exists in merged_df_with_valid_box_office
existing_ids = merged_df_with_valid_box_office['id'].tolist()
new_movies_df_filled = new_movies_df_filled[~new_movies_df_filled['id'].isin(existing_ids)]

# Append new_movies_df_filled to merged_df_with_valid_box_office
merged_df_with_valid_box_office = pd.concat([merged_df_with_valid_box_office, new_movies_df_filled], ignore_index=True)
merged_df_with_valid_box_office.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17725 entries, 0 to 17724
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  10220 non-null  object 
 1   belongs_to_collection  1852 non-null   object 
 2   homepage               2812 non-null   object 
 3   id                     17725 non-null  object 
 4   imdb_id                10220 non-null  object 
 5   production_countries   10220 non-null  object 
 6   spoken_languages       10220 non-null  object 
 7   title                  10220 non-null  object 
 8   video                  10220 non-null  object 
 9   keywords               14022 non-null  object 
 10  backdrop_path          15116 non-null  object 
 11  recommendations        9988 non-null   object 
 12  budget                 17719 non-null  object 
 13  genres                 17157 non-null  object 
 14  original_language      17725 non-null  object 
 15  or

In [70]:
# import ast

# def extract_genres(genres):
#     if isinstance(genres, str):
#         genres_list = ast.literal_eval(genres)
#         return [genre['name'] for genre in genres_list]
#     elif isinstance(genres, str):
#         return genres.split('-')
#     else:
#         return []

# merged_df_with_valid_box_office.genres = merged_df_with_valid_box_office.genres.apply(extract_genres)
# merged_df_with_valid_box_office.genres.head()


In [71]:
# test=merged_df_with_valid_box_office['genres'].tolist()


In [72]:
merged_df_with_valid_box_office.to_csv('data_output.csv', index=False)